In [50]:
#
# Public elliptic curve (sec512r1)

p = 2 ** 521 - 1
a = 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFC
b = 0x0051953EB9618E1C9A1F929A21A0B68540EEA2DA725B99B315F3B8B489918EF109E156193951EC7E937B1652C0BD3BB1BF073573DF883D2C34F1EF451FD46B503F00
E = EllipticCurve(FiniteField(p), [a, b])

x = 0x00C6858E06B70404E9CD9E3ECB662395B4429C648139053FB521F828AF606B4D3DBAA14B5E77EFE75928FE1DC127A2FFA8DE3348B3C1856A429BF97E7E31C2E5BD66
y = 0x011839296A789A3BC0045C8A5FB42C7D1BD998F54449579B446817AFBD17273E662C97EE72995EF42640C550B9013FAD0761353C7086A272C24088BE94769FD16650
G = E(x, y)

n = 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFA51868783BF2F966B7FCC0148F709A5D03BB5C9B8899C47AEBB6FB71E91386409

Fn = FiniteField(n)
#print(int(n).bit_length())

#(n * G).is_zero()

In [51]:
import os
import hashlib

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac

#
# Common functions

def i2b(i):
    """
    int to bytes
    """
    return int(i).to_bytes((int(i).bit_length() + 7) // 8, byteorder=sys.byteorder)

def b2i(b):
    """
    bytes to int
    """
    return int.from_bytes(b, byteorder=sys.byteorder)

def ecp2b(p):
    """
    elliptic curve point to bytes
    """
    x, y = p.xy()
    return i2b(int(x)) + i2b(int(y))

def h(m):
    """
    The H function (SHA-256)
    """
    return hashlib.sha256(m).digest()

def hp(m):
    """
    The H' function
    """
    return int.from_bytes(h(m), byteorder=sys.byteorder) * G

def auth_enc(key, message):
    
    # pad message with 0 as described in 3.1.1.
    message += b'\x00' * 16
    #while len(message) % 16 != 0:
    #    message += b'\x00'
    
    # iv should be 0 according to RFC
    iv = b'\x00' * 12  # os.urandom(12)
    
    # return iv and cipher
    return AESGCM(key).encrypt(iv, message, None)

def auth_dec(key, cipher):
    
    # iv should be 0 according to RFC
    iv = b'\x00' * 12  # os.urandom(12)
    
    # return message after removing the last 16 "0" bytes
    return AESGCM(key).decrypt(iv, cipher, None)[:-16]

def e(pub, id, ssid):
    return h(pub, id, ssid)

def key_ex_s(p_s, x_s, X_s, P_u, X_u, id_s, id_u, ssid):
    
    e_u = h(ecp2b(X_u) + id_s + ssid)
    e_s = h(ecp2b(X_s) + id_u + ssid)
    
    return h(ecp2b((X_u + b2i(e_u) * P_u) * (x_s + b2i(e_s) * p_s)))

def key_ex_u(p_u, x_u, X_s, P_s, X_u, id_s, id_u, ssid):
    
    e_u = h(ecp2b(X_u) + id_s + ssid)
    e_s = h(ecp2b(X_s) + id_u + ssid)
    
    return h(ecp2b((X_s + b2i(e_s) * P_s) * (x_u + b2i(e_u) * p_u)))

def f(key, message):
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(message)
    return h.finalize()

def abort():
    sys.exit(-1)
    None

#k = AESGCM.generate_key(bit_length=int(128))
#c = auth_enc(k, b'hello')
#print(c)
#print(auth_dec(k, c))

In [72]:
#
# Step 1. User

pw = b'pwd123'

# choose random r and x_u
r = Integer(Fn.random_element())
x_u = Integer(Fn.random_element())

# compute alpha and X_u
alpha = hp(pw) * r
X_u = x_u * G

5901424928663745546082807237476412072779202891153333495630275243529126635895359024705726865577831220639775694029036084082833563339161786604804804451360028385

User sends $\alpha$ and $X_u$ to the server.

In [53]:
#
# Step 2. Server

c = b'\xdbB7[\x0c\xf0\xf9\xf8\xb4Z\xbb\x19:\xf9\x05\x17\xf0q\x89\xf1NJ\x1f\x10}z\xfd\xf0X,\x9e\xf0\xf46\xcd\x9b^\xbf\xbf.\x03\xe5A\x8d\x18\xa0[\x8dd\x01B~ZN\xf9\x88wS"LZ8\x10\xc7\xcb\x15q\x98\xc2*\xd0\x06\x9d\xb0H\xb8+\xefb\xcb\xa1\xd7g|i\xc7\xa6\xd0\x1f\xb7\x17\xf3\x8d\xdd\x1d\xd7[\xfa{\x9bU\x18\x05\xb8\x165\xa3\xcd\x16\x13T\xe3\x19\x1ex\xce\xeb\xc5\xce)\xdf\x8aw+\xce\xb7\x05\x84\x85\xddz\xc2\xbe)is\xbd\x08Sr\xc7\xb2g8\xa2\xcczm4\xa8\xc1\x9d\xb1\xdb\x9e%\xffsRC\xba\x9b"J\r@eI\xff^\xa7\x8a\xc9*\x1c\xdac\x9al\xdb\xee\xaa\xd7\xc1$L:\xbb\xd4\x87\xa1\xde\x03/\xf1\x11v\xc1\xc5\x9c\x0c\x1f\xfa\xa3\xec\x96m\xa5\xa3\\\xc7!-\x117\xa7u\xb3g\xd6\xda\xad\xbbD\xce\xc7u\xe4\xa5\xb3\xd4\x03S\xaa_\xb77\xf0R\xcc\x1b\xef\xe2\xa7\x13p\xb3\xc0\xdd\x06\x8cp\x8f\t\x7f\xd0\xdb\x94\x85\x04\x16{\xe5\xa2\x12\xf2\xca\xd9\xd6\xe7\xcc\xed\x02\xfa\xa1\xba\xe2U\xdb$\xd4\x8a\x1f2\xabV\xa9\x85~\xb0~*Ot\x012^\x15G !\xb5z\x17\t\xd9\x8d\xeb\x12D"GO\xe5\x1fg\xfc\x8a\x88\xacX\xe4eJ\xf1_{\xf7r\x96\xad\xfcK\x86\xb3J\x98q\xd98\xae\xf3w%\x95\xa8\xb5\xect!\x0f\x95\xe47\xf8x\x8a\xdc\xa7 p'
pub_s = E(1433704665742155038128462656064416442871785350599550562436398511083464180941909401851713430314432670788559973193195534726546972726910093828042000496883413045, 6756872519399264193144605465473237174198164156820665282509453279257094869006302479753383364803613681545608262068774045368877452023372312138889429496436952089)
prv_s = 4205577181500221178842363737043702983229732488496938327623072198775824879650558028075099794835837376090085027129870661999331302921101231436690046483622785452
pub_u = E(1433704665742155038128462656064416442871785350599550562436398511083464180941909401851713430314432670788559973193195534726546972726910093828042000496883413045, 6756872519399264193144605465473237174198164156820665282509453279257094869006302479753383364803613681545608262068774045368877452023372312138889429496436952089)
k_s = 940506098320079733993637645689320100120596263903254754542617893037568640189179889095339283415665838876732151345027733024922018722143604368347926122043755291
id_s = b'srv'
id_u = b'usr'
ssid = b'123'
sid = b'usr'

# check alpha belongs to the curve
x, y = alpha.xy()
if not E.is_on_curve(x, y):
    abort()

# choose x_s
x_s = Integer(Fn.random_element())

# compute beta and and X_s
beta = alpha * k_s
X_s = x_s * G

# compute K, ssid', SK and A_s
K = key_ex_s(prv_s, x_s, X_s, pub_u, X_u, id_s, id_u, ssid)

ssidp = h(sid + ssid + ecp2b(alpha))
sk = f(K, b'\x00' + ssidp)
A_s = f(K, b'\x01' + ssidp)

Server sends $\beta$, $X_s$, $c$ and $A_s$ to the user.

In [74]:
#
# Step 3. User

# check beta belongs to the curve
x, y = alpha.xy()
if not E.is_on_curve(x, y):
    abort()

# compute rw and decrypt c
rw = h(pw + ecp2b(beta * r.inverse_mod(n)))
print(rw)
prv_u, pub_u, pub_s = auth_dec(rw, c)

# compute K, ssid', SK
K = key_ex_u(prv_u, x_u, pub_s, X_s)
ssidp = h(sid + ssid + ecp2b(alpha))
sk = f(K, b'\x00' + ssidp)

# compute A_s and verify it equals the one received from the server
if A_s != f(K, b'\x01' + ssidp):
    abort()

# compute A_u
A_u = f(K, b'\x02' + ssidp)

b'\x8f\x98\xad\xbe\xc6\x8a\x9e\xa3\xaaY\x85\x030\x8b\xd6\xf4|\xce\xafd\x9c\xa2\x16\x81lPT\x16\xf8\x19\x98\xf9'


InvalidTag: 

User sends $A_u$ to the server.

In [ ]:
#
# Step 4. Server

# compute A_u and verify it equals the one received from the user
if A_u != f(K, b'\x02' + ssidp):
    abort()